In [ ]:
!pip install SimpleITK tifffile matplotlib
!pip install aicsimageio
!pip install aicspylibczi

import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import dask.array as da
from aicsimageio import AICSImage


from google.colab import drive
drive.mount('/content/drive')




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 48.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of ome-zarr to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### functions


In [ ]:
from scipy.ndimage import zoom

def downsample_volume(volume, z_factor=0.5, xy_factor=0.5, order=1):
    """
    Downsample a 3D volume (Z, Y, X) by given factors.

    Parameters
    ----------
    volume : np.ndarray
        3D image (Z, Y, X) to downsample.
    z_factor : float
        Scaling factor along Z (e.g. 0.5 = half as many slices).
    xy_factor : float
        Scaling factor along Y and X (e.g. 0.25 = 4× smaller laterally).
    order : int
        Interpolation order: 0=nearest, 1=linear, 3=cubic.

    Returns
    -------
    small : np.ndarray
        Downsampled 3D volume.
    """
    assert volume.ndim == 3, "Input must be 3D (Z, Y, X)"
    zoom_factors = (z_factor, xy_factor, xy_factor)
    small = zoom(volume, zoom_factors, order=order)
    return small.astype(np.float32)



    # === 2. Load and normalize between 0–1 ===
def load_czi(path):
    # Load image with AICSImageIO
    img = AICSImage(path)

    data = img.get_image_data()

    # typical shape: (1,1,1,Z,Y,X,1)
    data = np.squeeze(data)
    # normalize 0–1
    data = data.astype(np.float32)
    data = (data - data.min()) / (data.max() - data.min() + 1e-8)
    return data

### running

In [ ]:
import os

# === 1. Define file paths ===
fixed_path  = r"/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data/2ii4_lifeactGFP_50pc_5ms-01.czi"
moving_path = r"/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data/2ii4_lifeactGFP_50pc_50ms-01.czi"
dir_path = os.path.dirname(moving_path)
name     = os.path.basename(moving_path[:-4])
print(dir_path)

fixed_np_full  = load_czi(fixed_path)
print(fixed_np_full.dtype)
print(fixed_np_full.shape)
imwrite(dir_path + "/registered/" + name + "5ms_fixed.tif", fixed_np_full.astype(np.float32))
print("💾 Saved as: fixed.tif")
moving_np_full = load_czi(moving_path)

z_factor  = 0.5
xy_factor = 0.5
fixed_np_down  = downsample_volume(fixed_np_full , z_factor=z_factor, xy_factor=xy_factor, order=1)
moving_np_down = downsample_volume(moving_np_full, z_factor=z_factor, xy_factor=xy_factor, order=1)

# Convert numpy arrays to SimpleITK images
fixed_sitk_full  = sitk.GetImageFromArray(fixed_np_full)
moving_sitk_full = sitk.GetImageFromArray(moving_np_full)
fixed_sitk_down  = sitk.GetImageFromArray(fixed_np_down)
moving_sitk_down = sitk.GetImageFromArray(moving_np_down)

# Create a plain 3D translation transform
initial_tx = sitk.TranslationTransform(3)

# Set up registration
reg = sitk.ImageRegistrationMethod()
reg.SetMetricAsCorrelation()
reg.SetInterpolator(sitk.sitkLinear)
reg.SetOptimizerAsRegularStepGradientDescent(
    learningRate=1.0, minStep=1e-4, numberOfIterations=200
)
reg.SetInitialTransform(initial_tx, inPlace=False)

print("🔄 Running translation-only registration...")
final_tx = reg.Execute(fixed_sitk_down, moving_sitk_down)
dz, dy, dx = final_tx.GetParameters()
full_translation = [dz * 1/z_factor, dy * 1/xy_factor, dx * 1/xy_factor]
print("✅ Translation (Z, Y, X):", full_translation)
tx_full = sitk.TranslationTransform(3)
tx_full.SetParameters(full_translation)


# Apply transform
registered_sitk = sitk.Resample(moving_sitk_full, fixed_sitk_full, tx_full, sitk.sitkLinear, 0.0, moving_sitk_full.GetPixelID())
registered_np = sitk.GetArrayFromImage(registered_sitk)
print(registered_np.shape)

imwrite(dir_path + "/registered/ " + name + "_5ms_registered.tif", registered_np.astype(np.float32))
print("💾 Saved as: pos_registered_translation.tif")

# === 5. Quick visual check (middle slice overlay) ===
z = fixed_np_full.shape[0] // 2
overlay = np.stack([fixed_np_full[z], registered_np[z], np.zeros_like(fixed_np_full[z])], axis=-1)

plt.figure(figsize=(10,4))
plt.subplot(1,3,1); plt.imshow(fixed_np_full[z], cmap='gray'); plt.title("Fixed"); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(registered_np[z], cmap='gray'); plt.title("Registered"); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(overlay); plt.title("Overlay"); plt.axis('off')
plt.show()

/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data
float32
(571, 759, 2048)
💾 Saved as: fixed.tif
🔄 Running translation-only registration...


KeyboardInterrupt: 